In [100]:
import numpy as np
import math
from itertools import combinations

np.set_printoptions(suppress=True, linewidth=np.nan)


In [101]:
def build_tableau(A: np.ndarray, b: np.ndarray, c: np.ndarray) -> np.ndarray:
    tableau = np.r_[[c], A]
    tableau = np.c_[tableau, np.insert(b, 0, 0)]
    return tableau


In [111]:
def minimum_ratio_test(col: np.ndarray, b: np.ndarray) -> int:
    r_min = 0
    min_val = math.inf
    for k in range(len(b)):
        if col[k] > 0:
            if min_val > (min_val_temp := b[k] / col[k]):
                min_val = min_val_temp
                r_min = k
    return r_min if min_val != math.inf else None


In [112]:
def pivoting(tableau: np.ndarray, row: int, col: int) -> np.ndarray:
    # escale pivot row min to 1.0
    tableau[row] = tableau[row] / tableau[row, col]
    # pivot proccess: convert al column to zero except row
    for k in range(len(tableau)):
        if k != row:
            tableau[k] = tableau[k] - tableau[k, col] * tableau[row, :]
    return tableau


In [113]:
def correct_tableau(tableau: np.ndarray, basic_var: list[int]):
    for index, col in enumerate(basic_var):
        row = index + 1
        tableau[row] = tableau[row] / tableau[row, col]
        for k in range(len(tableau)):
            if k != row:
                tableau[k] = tableau[k] - tableau[k, col] * tableau[row, :]
    return tableau


In [114]:
def basic_var2vertex(tableau: np.ndarray, basic_var: list[int]) -> None:
    # only work with simplex method because m = len(basic_var) and n = len(non_basic_var)
    dim_1, dim_2 = tableau.shape
    m = len(basic_var)
    n = dim_2 - dim_1  # for Big-M don't delete artificial variables
    x = np.zeros(m + n)
    x[basic_var] = tableau[1:, -1]
    return x[:n]


In [115]:
def real_index(basic_var: list[int]):
    return list(map(lambda x: x + 1, basic_var))


In [120]:
def simplex(tableau: np.ndarray, basic_var: list[int]) -> np.ndarray:
    path: list = [basic_var[:]]
    print(f"x_B = {real_index(basic_var)}\n{tableau}")
    while (tableau[0, :-1] > 0).any():
        # max criterion
        c_max = np.argmax(tableau[0, :-1])
        # minimum ratio test

        if (r_min := minimum_ratio_test(col=tableau[1:, c_max], b=tableau[1:, -1])) != None:
            r_min += 1
            # pivoting
            tableau = pivoting(tableau, r_min, c_max)
            # swap row with col
            basic_var[r_min - 1] = c_max

            vertex = basic_var2vertex(tableau, basic_var)
            print(f"vertex = {vertex}\tx_B = {real_index(basic_var)}\n{tableau}\npivoted = {r_min, c_max}")

            if basic_var in path:
                path.append(basic_var[:])
                print(f"There's a cycle {basic_var}")
                break
            path.append(basic_var[:])
        else:
            print("The model is unbounded")
            break

    print(list(map(lambda x: real_index(x), path)))
    return tableau


In [121]:
A = [[1/2, -11/2, -5/2, 9, 1, 0],
     [1/2, -3/2, -1/2, 1., 0, 1]]

b = [0.0, 0.0]

c = [10, -57, -9, -24, 0.0, 0.0]

m = len(b)
n = len(c) - m
tableau = build_tableau(A, b, c)
all_basic_var = map(lambda x: list(x), combinations(range(n + m), m))
for basic_var in all_basic_var:
     tableau = correct_tableau(tableau, basic_var)
     print("------- New simplex proccess -------")
     simplex(tableau, basic_var)
     print("End process")



# basic_var = [4, 5]
# tableau = build_tableau(A, b, c)
# simplex(tableau, basic_var)


------- New simplex proccess -------
x_B = [1, 2]
[[  0.     0.    14.5  -98.    -6.75 -13.25   0.  ]
 [  1.     0.     0.5   -4.    -0.75   2.75   0.  ]
 [  0.     1.     0.5   -2.    -0.25   0.25   0.  ]]
vertex = [0. 0. 0. 0.]	x_B = [3, 2]
[[-29.    0.    0.   18.   15.  -93.    0. ]
 [  2.    0.    1.   -8.   -1.5   5.5   0. ]
 [ -1.    1.    0.    2.    0.5  -2.5   0. ]]
pivoted = (1, 2)
vertex = [0. 0. 0. 0.]	x_B = [3, 4]
[[-20.    -9.     0.     0.    10.5  -70.5    0.  ]
 [ -2.     4.     1.     0.     0.5   -4.5    0.  ]
 [ -0.5    0.5    0.     1.     0.25  -1.25   0.  ]]
pivoted = (2, 3)
vertex = [0. 0. 0. 0.]	x_B = [5, 4]
[[ 22.  -93.  -21.    0.    0.   24.    0. ]
 [ -4.    8.    2.    0.    1.   -9.    0. ]
 [  0.5  -1.5  -0.5   1.    0.    1.    0. ]]
pivoted = (1, 4)
vertex = [0. 0. 0. 0.]	x_B = [5, 6]
[[ 10.  -57.   -9.  -24.    0.    0.    0. ]
 [  0.5  -5.5  -2.5   9.    1.    0.    0. ]
 [  0.5  -1.5  -0.5   1.    0.    1.    0. ]]
pivoted = (2, 5)
vertex = [0. 0. 

In [118]:
A = [[1 / 3, 1, -1 / 3, -2, 1, 0],
     [-2.0, -9.0, 1.0, 9.0, 0, 1]]

b = [0.0, 0.0]

c = [2, 3, -1, -12, 0, 0.0]

m = len(b)
n = len(c) - m
tableau = build_tableau(A, b, c)
all_basic_var = map(lambda x: list(x), combinations(range(n + m), m))
for basic_var in all_basic_var:
     tableau = correct_tableau(tableau, basic_var)
     print("Start process")
     simplex(tableau, basic_var)
     print("End process")



# basic_var = [4, 5]
# tableau = build_tableau(A, b, c)
# simplex(tableau, basic_var)


Start process
x_B = [1, 2]
[[  0.           0.           2.           3.         -12.          -1.           0.        ]
 [  1.           0.          -2.          -9.           9.           1.           0.        ]
 [ -0.           1.           0.33333333   1.          -2.          -0.33333333  -0.        ]]
vertex = [ 0.  0.  0. -0.]	x_B = [1, 4]
[[ 0.         -3.          1.          0.         -6.          0.          0.        ]
 [ 1.          9.          1.          0.         -9.         -2.          0.        ]
 [-0.          1.          0.33333333  1.         -2.         -0.33333333 -0.        ]]
pivoted = (2, 3)
vertex = [ 0.  0.  0. -0.]	x_B = [3, 4]
[[ -1.         -12.           0.           0.           3.           2.           0.        ]
 [  1.           9.           1.           0.          -9.          -2.           0.        ]
 [ -0.33333333  -2.           0.           1.           1.           0.33333333  -0.        ]]
pivoted = (1, 2)
vertex = [0. 0. 0. 0.]	x_B = [3

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [110]:
6*5/2

15.0